In [115]:
import os 
import json
import pandas as pd
import traceback

In [116]:
from langchain_openai import ChatOpenAI

In [117]:
from dotenv import load_dotenv
load_dotenv() # take enviroment variable from .env

True

In [118]:
mykey = os.getenv("OPENAI_API_KEY")

In [119]:
llm = ChatOpenAI(openai_api_key = mykey, model = 'gpt-4o', temperature=0.2)

In [120]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001AEE382E220>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001AEE382AD60>, root_client=<openai.OpenAI object at 0x000001AEE3702CA0>, root_async_client=<openai.AsyncOpenAI object at 0x000001AEE382EB50>, model_name='gpt-4o', temperature=0.2, openai_api_key=SecretStr('**********'), openai_proxy='')

In [121]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [122]:
RESPONSE_JSON = {
    '1': {
        'mcq':'multiple choices question',
        'options': {
            'a':"choice here",
            'a':"choice here",
            'a':"choice here",
            'a':"choice here",
        },
        'correct':"correct answer"
    },
    '2':{
        'mcq':"multiple choices question",
        'option':{
            'a':'choices here',
            'b':'choices here',
            'c':'choices here',
            'd':'choices here',
        },
        'correct':'correct answer'
    },
    '3':{
        'mcq':'multiple choices question',
        'option':{
            'a':'choices here',
            'b':'choices here',
            'c':'choices here',
            'd':'choices here',
        },
        'correct':'correct answer'
    },
    '4':{
        'mcq':'multiple choices question',
        'option':{
            'a':'choices here',
            'b':'choices here',
            'c':'choices here',
            'd':'choices here',
        },
        'correct':'correct answer'
}
}

In [123]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs and Return only JSON, without any additional text or Markdown code blocks (```).
### RESPONSE_JSON
{response_json}

"""

In [124]:
quiz_generation_prompt = PromptTemplate(
    input_variables= ['text', 'number', 'subject', 'tone','response_json'],
    template = TEMPLATE
)

In [125]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [126]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [127]:
quiz_evaluation_prompt = PromptTemplate(input_variables=['subject','quiz'],
                                        template= TEMPLATE2)

In [128]:
review_chain = LLMChain(llm=llm, prompt = quiz_evaluation_prompt, output_key = 'review', verbose = True)

In [129]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [130]:
file_path = r"C:\Users\LEX\mcqgen\data.txt"

In [131]:
with open(file_path, 'r') as file:
    TEXT = file.read()

TEXT = TEXT.lstrip("\ufeff")
print(TEXT)

As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.[22] Probabilistic reasoning was also employed, especially in automated medical diagnosis.[23]: 488 

However, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.[23]: 488  By 1980, expert systems had come to dominate AI, and statistics was out of favor.[24] Work on symbolic/knowledge-based learning did continue within AI, leading to inductive logic 

In [132]:
# Dic into Json-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choices question", "options": {"a": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choices question", "option": {"a": "choices here", "b": "choices here", "c": "choices here", "d": "choices here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choices question", "option": {"a": "choices here", "b": "choices here", "c": "choices here", "d": "choices here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choices question", "option": {"a": "choices here", "b": "choices here", "c": "choices here", "d": "choices here"}, "correct": "correct answer"}}'

In [133]:
NUMBER = 5,
SUBJECT = "machine learning"
TONE = "simple"

In [134]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain.invoke(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json":json.dumps(RESPONSE_JSON)
        }
        
    )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.[22] Probabilistic reasoning was also employed, especially in automated medical diagnosis.[23]: 488 

However, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.[23]: 488  By 1980, expert systems had come to dominate AI, and statistics was out of favor.[24] Work on symbolic/knowledge-based learning did continue within 

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for machine learning students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{"1": {"mcq": "What was the initial goal of machine learning as a scientific endeavor?", "options": {"a": "To develop expert systems", "b": "To achieve artificial intelligence", "c": "To improve data representation", "d": "To enhance symbolic methods"}, "correct": "b"}, "2": {"mcq": "Which approach caused a rift between AI and machine learning?", "options": {"a": "Neural networks", "b": "Probabilistic reasoning", "c": "Logical, knowledge-based approach", "d": "Pattern recognitio

In [135]:
print(cb.total_tokens)
print(cb.prompt_tokens)
print(cb.completion_tokens)
print(cb.total_cost)

1962
1140
822
0.01107


In [136]:
response

{'text': 'As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.[22] Probabilistic reasoning was also employed, especially in automated medical diagnosis.[23]:\u200a488\u200a\n\nHowever, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.[23]:\u200a488\u200a By 1980, expert systems had come to dominate AI, and statistics was out of favor.[24] Work on symbolic/knowledge-based learning did continue within

In [142]:
quiz = response.get('quiz')
print(quiz)

{"1": {"mcq": "What was the initial goal of machine learning as a scientific endeavor?", "options": {"a": "To develop expert systems", "b": "To achieve artificial intelligence", "c": "To improve data representation", "d": "To enhance symbolic methods"}, "correct": "b"}, "2": {"mcq": "Which approach caused a rift between AI and machine learning?", "options": {"a": "Neural networks", "b": "Probabilistic reasoning", "c": "Logical, knowledge-based approach", "d": "Pattern recognition"}, "correct": "c"}, "3": {"mcq": "What dominated AI by 1980?", "options": {"a": "Neural networks", "b": "Expert systems", "c": "Inductive logic programming", "d": "Pattern recognition"}, "correct": "b"}, "4": {"mcq": "Who were some of the researchers involved in continuing neural networks research outside AI?", "options": {"a": "John Hopfield, David Rumelhart, Geoffrey Hinton", "b": "Alan Turing, John McCarthy, Marvin Minsky", "c": "Andrew Ng, Yann LeCun, Yoshua Bengio", "d": "Herbert Simon, Allen Newell, Arth

In [143]:
print(f"quiz 的长度: {len(quiz)}")
print(f"quiz 内容（前100个字符）: {quiz[:100]}")


quiz 的长度: 1204
quiz 内容（前100个字符）: {"1": {"mcq": "What was the initial goal of machine learning as a scientific endeavor?", "options": 


In [146]:
quiz  = json.loads(quiz)

In [147]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [148]:
quiz_table_data

[{'MCQ': 'What was the initial goal of machine learning as a scientific endeavor?',
  'Choices': 'a: To develop expert systems | b: To achieve artificial intelligence | c: To improve data representation | d: To enhance symbolic methods',
  'Correct': 'b'},
 {'MCQ': 'Which approach caused a rift between AI and machine learning?',
  'Choices': 'a: Neural networks | b: Probabilistic reasoning | c: Logical, knowledge-based approach | d: Pattern recognition',
  'Correct': 'c'},
 {'MCQ': 'What dominated AI by 1980?',
  'Choices': 'a: Neural networks | b: Expert systems | c: Inductive logic programming | d: Pattern recognition',
  'Correct': 'b'},
 {'MCQ': 'Who were some of the researchers involved in continuing neural networks research outside AI?',
  'Choices': 'a: John Hopfield, David Rumelhart, Geoffrey Hinton | b: Alan Turing, John McCarthy, Marvin Minsky | c: Andrew Ng, Yann LeCun, Yoshua Bengio | d: Herbert Simon, Allen Newell, Arthur Samuel',
  'Correct': 'a'},
 {'MCQ': 'In which deca

In [150]:
quiz  = pd.DataFrame(quiz_table_data)

In [151]:
quiz.to_csv("machine learning.csv", index=True)

In [1]:
from datetime import datetime
datetime.now()

datetime.datetime(2025, 2, 26, 12, 21, 25, 355625)